In [2]:

import openpyxl
from openpyxl import Workbook
from openpyxl.styles import Font, Alignment, PatternFill, Border, Side, Protection
from openpyxl.worksheet.datavalidation import DataValidation
from openpyxl.worksheet.protection import SheetProtection
from openpyxl.utils import get_column_letter

def generate_expense_list_excel_v7_detailed():
    """
    生成 V7 版交屋租金分算表
    更新重點：
    1. 跨月計算詳情顯示具體算式 (如 15000*3/30 + 15000*30/31)
    2. 賣方費用欄位擴充至 8 項，與買方對稱
    3. 買方補貼賣方邏輯同步更新
    """
    wb = Workbook()
    wb.remove(wb.active)
    ws = wb.create_sheet('交屋分算(費用明細)', 0)
    ws.sheet_view.showGridLines = False

    # --- 1. 樣式與設定 ---
    ws.page_setup.paperSize = ws.PAPERSIZE_A4
    ws.page_setup.orientation = ws.ORIENTATION_LANDSCAPE
    ws.page_setup.fitToWidth = 1
    ws.page_setup.fitToHeight = 0
    ws.page_margins.left = 0.25
    ws.page_margins.right = 0.25
    ws.page_margins.top = 0.5
    ws.page_margins.bottom = 0.5
    ws.print_title_rows = '7:7'

    # 顏色
    COLOR_HEADER_BG = "1E293B"
    COLOR_INPUT_BG = "FFFFFF"
    COLOR_SELLER_BG = "FFF1F2"
    COLOR_SELLER_HEADER = "E11D48"
    COLOR_SELLER_TXT = "881337"
    COLOR_BUYER_BG = "ECFDF5"
    COLOR_BUYER_HEADER = "059669"
    COLOR_BUYER_TXT = "064E3B"
    COLOR_FINAL_BG = "FFEDD5"
    COLOR_BORDER = "CBD5E1"
    COLOR_BORDER_THICK = "334155"
    COLOR_SIG_BG = "F8FAFC"
    COLOR_SUBTOTAL_BG = "F1F5F9"

    # 字型
    font_header_main = Font(name='微軟正黑體', size=18, bold=True, color='1E293B')
    font_header_base = Font(name='微軟正黑體', size=11, bold=True, color='FFFFFF')
    font_normal = Font(name='微軟正黑體', size=11, color='334155')
    font_input = Font(name='微軟正黑體', size=11, color='0F172A', bold=True)
    font_hint = Font(name='微軟正黑體', size=9, color='64748B')
    font_seller = Font(name='微軟正黑體', size=10, color=COLOR_SELLER_TXT, bold=True)
    font_buyer = Font(name='微軟正黑體', size=10, color=COLOR_BUYER_TXT, bold=True)
    font_subtotal = Font(name='微軟正黑體', size=11, bold=True, color='1E293B')
    font_big_res = Font(name='微軟正黑體', size=14, color='9A3412', bold=True)

    align_c = Alignment(horizontal='center', vertical='center', wrap_text=True)
    align_l = Alignment(horizontal='left', vertical='center', wrap_text=True)
    align_r = Alignment(horizontal='right', vertical='center', wrap_text=True)
    align_top_l = Alignment(horizontal='left', vertical='top', wrap_text=True)

    border_std = Border(
        left=Side(style='thin', color=COLOR_BORDER),
        right=Side(style='thin', color=COLOR_BORDER),
        top=Side(style='thin', color=COLOR_BORDER),
        bottom=Side(style='thin', color=COLOR_BORDER)
    )
    border_thick_right = Border(
        left=Side(style='thin', color=COLOR_BORDER),
        right=Side(style='medium', color=COLOR_BORDER_THICK),
        top=Side(style='thin', color=COLOR_BORDER),
        bottom=Side(style='thin', color=COLOR_BORDER)
    )
    border_thick_side = Side(style='thick', color='334155')

    def set_outer_border(ws, row_start, row_end, col_start, col_end):
        for c in range(col_start, col_end + 1):
            cell = ws.cell(row=row_start, column=c)
            current = cell.border
            cell.border = Border(left=current.left, right=current.right, top=border_thick_side, bottom=current.bottom)
        for c in range(col_start, col_end + 1):
            cell = ws.cell(row=row_end, column=c)
            current = cell.border
            cell.border = Border(left=current.left, right=current.right, top=current.top, bottom=border_thick_side)
        for r in range(row_start, row_end + 1):
            cell = ws.cell(row=r, column=col_start)
            current = cell.border
            cell.border = Border(left=border_thick_side, right=current.right, top=current.top, bottom=current.bottom)
        for r in range(row_start, row_end + 1):
            cell = ws.cell(row=r, column=col_end)
            current = cell.border
            cell.border = Border(left=current.left, right=border_thick_side, top=current.top, bottom=current.bottom)

    def set_cell(coord, value, font=font_normal, fill=None, align=align_c, border=border_std, fmt=None):
        c = ws[coord]
        c.value = value
        c.font = font
        c.alignment = align
        if border: c.border = border
        if fill: c.fill = PatternFill('solid', fgColor=fill)
        if fmt: c.number_format = fmt

    # 欄寬
    ws.column_dimensions['A'].width = 16
    ws.column_dimensions['B'].width = 12
    ws.column_dimensions['C'].width = 12
    ws.column_dimensions['D'].width = 13
    ws.column_dimensions['E'].width = 13
    ws.column_dimensions['F'].width = 12
    ws.column_dimensions['G'].width = 32 # 加寬以容納長算式
    ws.column_dimensions['H'].width = 13
    ws.column_dimensions['I'].width = 32 # 加寬以容納長算式
    ws.column_dimensions['J'].width = 13

    # --- 2. 標頭區 ---
    set_cell("A1", "交屋租金分算表", font=font_header_main, align=align_l, border=None)
    ws.merge_cells("A1:J1")
    ws.row_dimensions[1].height = 35

    set_cell("A2", "買方:", font=font_header_base, fill=COLOR_HEADER_BG, align=align_r)
    set_cell("B2", "陳小明", font=font_input, fill=COLOR_INPUT_BG, align=align_l)
    ws.merge_cells("B2:E2")
    set_cell("F2", "賣方:", font=font_header_base, fill=COLOR_HEADER_BG, align=align_r)
    set_cell("G2", "林大華", font=font_input, fill=COLOR_INPUT_BG, align=align_l)
    ws.merge_cells("G2:J2")
    ws.row_dimensions[2].height = 25

    set_cell("A3", "房屋地址:", font=font_header_base, fill=COLOR_HEADER_BG, align=align_r)
    set_cell("B3", "台北市信義區信義路五段7號", font=font_input, fill=COLOR_INPUT_BG, align=align_l)
    ws.merge_cells("B3:J3")
    ws.row_dimensions[3].height = 25

    set_cell("A4", "交屋日期 (7碼)", font=font_header_base, fill=COLOR_HEADER_BG)
    set_cell("B4", 1141127, font=font_input, fill=COLOR_INPUT_BG, fmt="0")
    verify_formula = f"""=IF(B4="","", TEXT(DATE(1911+LEFT(B4,3),MID(B4,4,2),RIGHT(B4,2)),"yyyy/mm/dd"))"""
    set_cell("C4", verify_formula, font=font_input, align=align_l, border=None)
    set_cell("D4", "(買賣雙方合意如租金有誤時，雙方另行找補)", font=Font(name='微軟正黑體', size=10, color='E11D48', bold=True), align=align_l, border=None)
    ws.merge_cells("D4:J4")
    ws.row_dimensions[4].height = 25

    set_cell("A5", "交屋日算誰的", font=font_header_base, fill=COLOR_HEADER_BG)
    set_cell("B5", "歸賣方", font=font_input, fill=COLOR_INPUT_BG)
    ws.merge_cells("C5:E5")
    set_cell("C5", '="⚖️ 權益切割：交屋日當天歸 " & RIGHT(B5,2)', font=font_hint, align=align_l, border=None)
    dv_owner = DataValidation(type='list', formula1='"歸買方,歸賣方"', allow_blank=False)
    ws.add_data_validation(dv_owner)
    dv_owner.add('B5:B5')
    ws.row_dimensions[5].height = 25

    set_cell("A6", "計算基準", font=font_header_base, fill=COLOR_HEADER_BG)
    set_cell("B6", "精確分段計算", font=font_normal, fill="F1F5F9")
    ws.merge_cells("C6:F6")
    calc_desc = '="📊 規則：若跨月，將依該月天數分拆計算 (例：$15000×3/30 + $15000×30/31)"'
    set_cell("C6", calc_desc, font=font_hint, align=align_l, border=None)
    ws.row_dimensions[6].height = 25

    # --- 3. 租金分算數據表 ---
    headers = ['房號/租客', '租期起', '租期訖', '月租金', '押金', '租客已繳至', '賣方補貼買方\n(計算詳情)', '補貼金額', '買方補貼賣方\n(計算詳情)', '補貼金額']
    ws.row_dimensions[7].height = 40
    for i, h in enumerate(headers):
        col = get_column_letter(i+1)
        bg_color = '64748B'
        if '賣方' in h: bg_color = COLOR_SELLER_HEADER
        if '買方' in h: bg_color = COLOR_BUYER_HEADER
        cell_border = border_std
        if col == 'F': cell_border = border_thick_right
        set_cell(f'{col}7', h, font=font_header_base, fill=bg_color, border=cell_border)

    start_row = 8
    end_row = 19

    # --- 核心邏輯 Helper Functions ---
    def get_prorated_rent_value_formula(rent_ref, s_date, e_date):
        """ 計算金額的公式 (保持精確) """
        cond_same = f"(EOMONTH({s_date},0)=EOMONTH({e_date},0))"
        factor_same = f"(({e_date}-{s_date}+1)/DAY(EOMONTH({s_date},0)))"
        p_start = f"((EOMONTH({s_date},0)-{s_date}+1)/DAY(EOMONTH({s_date},0)))"
        p_end = f"(DAY({e_date})/DAY(EOMONTH({e_date},0)))"
        p_mid = f"MAX(0, (YEAR({e_date})-YEAR({s_date}))*12 + MONTH({e_date})-MONTH({s_date}) - 1)"
        factor_cross = f"({p_start} + {p_end} + {p_mid})"
        return f"ROUND({rent_ref} * IF({cond_same}, {factor_same}, {factor_cross}), 0)"

    def get_prorated_rent_text_formula(rent_ref, s_date, e_date):
        """ 
        產生描述文字的公式 (視覺化算式)
        同月顯示: Date~Date (Days) \n $Rent * (Days/Total)
        跨月顯示: Date~Date (Total Days) \n $Rent * (D1/T1) + $Rent * (D2/T2) ...
        """
        # 同月邏輯
        days_same = f"({e_date}-{s_date}+1)"
        total_same = f"DAY(EOMONTH({s_date},0))"
        txt_same_math = f""" "$" & {rent_ref} & "×(" & {days_same} & "/" & {total_same} & ")" """
        txt_same = f""" TEXT({s_date}, "mm/dd") & "~" & TEXT({e_date}, "mm/dd") & " (" & {days_same} & "天)" & CHAR(10) & {txt_same_math} """

        # 跨月邏輯 (兩段式拆解，針對最常見的跨越兩個月情境)
        # 第一段 (首月): Start ~ EOM
        d1 = f"(EOMONTH({s_date},0)-{s_date}+1)"
        t1 = f"DAY(EOMONTH({s_date},0))"
        # 第二段 (尾月): StartOfEndMonth ~ End
        d2 = f"DAY({e_date})"
        t2 = f"DAY(EOMONTH({e_date},0))"
        
        # 顯示算式字串: $15000*(3/30) + $15000*(30/31)
        # 為了簡潔，我們不顯示中間日期拆分，直接顯示算式加總，這符合使用者需求
        txt_cross_math = f""" "$" & {rent_ref} & "×(" & {d1} & "/" & {t1} & ") + $" & {rent_ref} & "×(" & {d2} & "/" & {t2} & ")" """
        
        # 處理超過兩個月的狀況 (加註 "+...")
        is_multi = f"((YEAR({e_date})-YEAR({s_date}))*12 + MONTH({e_date})-MONTH({s_date}) > 1)"
        txt_cross_math_final = f""" IF({is_multi}, {txt_cross_math} & " +...", {txt_cross_math}) """

        days_total = f"({e_date}-{s_date}+1)"
        txt_cross = f""" TEXT({s_date}, "mm/dd") & "~" & TEXT({e_date}, "mm/dd") & " (" & {days_total} & "天)" & CHAR(10) & {txt_cross_math_final} """

        # 組合
        cond_same = f"(EOMONTH({s_date},0)=EOMONTH({e_date},0))"
        return f"IF({cond_same}, {txt_same}, {txt_cross})"

    for r in range(start_row, end_row + 1):
        row_str = str(r)
        ws.row_dimensions[r].height = 35

        # 初始化儲存格
        for c_char in ['A', 'B', 'C', 'D', 'E']:
            set_cell(f'{c_char}{row_str}', '', fill=COLOR_INPUT_BG)
        set_cell(f'F{row_str}', '', fill=COLOR_INPUT_BG, border=border_thick_right)

        ws[f'B{row_str}'].number_format = '0'
        ws[f'C{row_str}'].number_format = '0'
        ws[f'D{row_str}'].number_format = '#,##0'
        ws[f'E{row_str}'].number_format = '#,##0'
        ws[f'F{row_str}'].number_format = '0'

        # --- 日期變數 ---
        safe_b4 = f"IF($B$4=\"\", 1140101, $B$4)"
        d_handover = f"DATE(1911+LEFT({safe_b4},3),MID({safe_b4},4,2),RIGHT({safe_b4},2))"
        
        safe_f = f"IF(F{row_str}=\"\", {safe_b4}, F{row_str})"
        d_paid = f"DATE(1911+LEFT({safe_f},3),MID({safe_f},4,2),RIGHT({safe_f},2))"
        
        d_cut = f"IF($B$5=\"歸買方\", {d_handover}, {d_handover}+1)"
        
        is_valid = f'AND(D{row_str}<>"", F{row_str}<>"")'
        rent_ref = f"D{row_str}"

        # 1. 賣方補貼買方 (預收退還)
        # 期間: 切割日(d_cut) ~ 已繳至(d_paid)
        cond_seller = f"({d_paid} >= {d_cut})"
        s_val_formula = get_prorated_rent_value_formula(rent_ref, d_cut, d_paid)
        s_txt_formula = get_prorated_rent_text_formula(rent_ref, d_cut, d_paid)
        
        set_cell(f"G{row_str}", f"""=IF({is_valid}, IF({cond_seller}, {s_txt_formula}, ""), "")""", fill=COLOR_SELLER_BG, font=font_seller)
        ws[f"G{row_str}"].alignment = Alignment(wrap_text=True, horizontal="center", vertical="center")
        set_cell(f"H{row_str}", f"""=IF({is_valid}, IF({cond_seller}, {s_val_formula}, 0), "")""", fill=COLOR_SELLER_BG, fmt="#,##0", font=font_seller)

        # 2. 買方補貼賣方 (欠繳補償)
        # 期間: 已繳至+1 (d_paid+1) ~ 切割日-1 (d_cut-1)
        b_date_start = f"({d_paid}+1)"
        b_date_end = f"({d_cut}-1)"
        cond_buyer = f"({d_cut} > {d_paid} + 1)"
        
        b_val_formula = get_prorated_rent_value_formula(rent_ref, b_date_start, b_date_end)
        b_txt_formula = get_prorated_rent_text_formula(rent_ref, b_date_start, b_date_end)

        set_cell(f"I{row_str}", f"""=IF({is_valid}, IF({cond_buyer}, {b_txt_formula}, ""), "")""", fill=COLOR_BUYER_BG, font=font_buyer)
        ws[f"I{row_str}"].alignment = Alignment(wrap_text=True, horizontal="center", vertical="center")
        set_cell(f"J{row_str}", f"""=IF({is_valid}, IF({cond_buyer}, {b_val_formula}, 0), "")""", fill=COLOR_BUYER_BG, fmt="#,##0", font=font_buyer)

    # 範例資料
    set_cell("A8", "301 (範例)", fill=COLOR_INPUT_BG)
    set_cell("B8", 1141122, fill=COLOR_INPUT_BG, fmt="0")
    set_cell("C8", 1141221, fill=COLOR_INPUT_BG, fmt="0")
    set_cell("D8", 15000, fill=COLOR_INPUT_BG, fmt="#,##0")
    set_cell("E8", 30000, fill=COLOR_INPUT_BG, fmt="#,##0")
    set_cell("F8", 1141230, fill=COLOR_INPUT_BG, fmt="0", border=border_thick_right)

    # 加粗框
    set_outer_border(ws, 7, 19, 1, 10)

    # --- 4. 結算區 (對稱化) ---
    sum_start_row = end_row + 2
    ws.merge_cells(f'A{sum_start_row}:D{sum_start_row}')
    set_cell(f'A{sum_start_row}', '賣方應付項目 (Debits)', font=font_header_base, fill=COLOR_SELLER_HEADER, align=align_l)
    ws.merge_cells(f'F{sum_start_row}:J{sum_start_row}')
    set_cell(f'F{sum_start_row}', '買方應付項目及費用 (Credits)', font=font_header_base, fill=COLOR_BUYER_HEADER, align=align_l)
    ws.row_dimensions[sum_start_row].height = 30

    # 擴充賣方項目至 8 項
    seller_items = [
        ('1. 押金移轉', f'=SUM(E{start_row}:E{end_row})', False),
        ('2. 賣方補貼買方 (預收租金)', f'=SUM(H{start_row}:H{end_row})', False),
        ('3. 其他費用 (1)', 0, True),
        ('4. 其他費用 (2)', 0, True),
        ('5. 其他費用 (3)', 0, True),
        ('6. 其他費用 (4)', 0, True),
        ('7. 其他費用 (5)', 0, True),
        ('8. 其他費用 (6)', 0, True),
    ]
    buyer_items = [
        ('1. 買方補貼賣方 (欠繳租金)', f'=SUM(J{start_row}:J{end_row})', False),
        ('2. 機上盒保證金', 0, True),
        ('3. 有線電視費用', 0, True),
        ('4. 網路費用', 0, True),
        ('5. 監視器費用', 0, True),
        ('6. 寬頻押金', 0, True),
        ('7. 其他費用 (1)', 0, True),
        ('8. 其他費用 (2)', 0, True),
    ]

    max_len = max(len(seller_items), len(buyer_items))
    seller_amount_cells = []
    buyer_amount_cells = []
    start_item_row = sum_start_row + 1

    for i in range(max_len):
        r = start_item_row + i
        ws.row_dimensions[r].height = 25
        
        # Seller Side
        ws.merge_cells(f'A{r}:C{r}')
        if i < len(seller_items):
            name, val, is_input = seller_items[i]
            fill = COLOR_INPUT_BG if is_input else None
            set_cell(f'A{r}', name, align=align_l, border=None, fill=fill if "其他" in name else None)
            set_cell(f'D{r}', val, fmt='#,##0', font=font_seller, align=align_r, fill=COLOR_INPUT_BG if is_input else None)
            seller_amount_cells.append(f'D{r}')
        else:
            set_cell(f'A{r}', '', border=None)
            set_cell(f'D{r}', '', border=None)

        # Buyer Side
        ws.merge_cells(f'F{r}:I{r}')
        if i < len(buyer_items):
            name, val, is_input = buyer_items[i]
            fill = COLOR_INPUT_BG if is_input else None
            set_cell(f'F{r}', name, align=align_l, border=None, fill=fill if "其他" in name else None)
            set_cell(f'J{r}', val, fmt='#,##0', font=font_buyer, align=align_r, fill=COLOR_INPUT_BG if is_input else None)
            buyer_amount_cells.append(f'J{r}')
        else:
            set_cell(f'F{r}', '', border=None)
            set_cell(f'J{r}', '', border=None)

    # 小計
    subtotal_row = start_item_row + max_len
    ws.row_dimensions[subtotal_row].height = 30
    
    ws.merge_cells(f'A{subtotal_row}:C{subtotal_row}')
    set_cell(f'A{subtotal_row}', '賣方應付小計', font=font_subtotal, fill=COLOR_SUBTOTAL_BG, align=align_r)
    sum_seller_formula = "+".join(seller_amount_cells) if seller_amount_cells else "0"
    set_cell(f'D{subtotal_row}', f'={sum_seller_formula}', fmt='#,##0', font=font_subtotal, fill=COLOR_SUBTOTAL_BG, align=align_r)

    ws.merge_cells(f'F{subtotal_row}:I{subtotal_row}')
    set_cell(f'F{subtotal_row}', '買方應付小計', font=font_subtotal, fill=COLOR_SUBTOTAL_BG, align=align_r)
    sum_buyer_formula = "+".join(buyer_amount_cells) if buyer_amount_cells else "0"
    set_cell(f'J{subtotal_row}', f'={sum_buyer_formula}', fmt='#,##0', font=font_subtotal, fill=COLOR_SUBTOTAL_BG, align=align_r)

    # 結算框線
    set_outer_border(ws, sum_start_row, subtotal_row, 1, 4)
    set_outer_border(ws, sum_start_row, subtotal_row, 6, 10)

    # 最終淨額
    final_row = subtotal_row + 1
    ws.merge_cells(f'A{final_row}:J{final_row}')
    ws.row_dimensions[final_row].height = 50
    calc_cell = f'K{final_row}'
    ws[calc_cell] = f'=(D{subtotal_row}) - (J{subtotal_row})'
    text_formula = f"""=IF({calc_cell}>0, "最終結算：賣方應找補買方 $" & TEXT({calc_cell}, "#,##0"), IF({calc_cell}<0, "最終結算：買方應找補賣方 $" & TEXT(ABS({calc_cell}), "#,##0"), "最終結算：雙方無須找補"))"""
    set_cell(f'A{final_row}', text_formula, font=font_big_res, align=align_c, fill=COLOR_FINAL_BG)
    set_outer_border(ws, final_row, final_row, 1, 10)

    # 簽名區
    sig_row = final_row + 2
    ws.merge_cells(f'A{sig_row}:C{sig_row}')
    font_sig = Font(name='微軟正黑體', size=11, bold=True, color='1E293B')
    set_cell(f'A{sig_row}', "買方簽章：____________________", font=font_sig, fill=COLOR_SIG_BG, align=align_l)
    set_outer_border(ws, sig_row, sig_row, 1, 3)
    ws.merge_cells(f'F{sig_row}:H{sig_row}')
    set_cell(f'F{sig_row}', "賣方簽章：____________________", font=font_sig, fill=COLOR_SIG_BG, align=align_l)
    ws.row_dimensions[sig_row].height = 50
    set_outer_border(ws, sig_row, sig_row, 6, 8)

    # 備註
    rem_row = sig_row + 2
    ws.merge_cells(f'A{rem_row}:J{rem_row+4}')
    set_cell(f'A{rem_row}', "備註 / Remarks：", font=font_normal, align=align_top_l, border=border_thick_right)
    for r_rem in range(rem_row, rem_row + 5):
        for c_rem in range(1, 11):
            ws.cell(row=r_rem, column=c_rem).border = border_std
    ws[f'A{rem_row}'].value = "備註 / Remarks："
    ws[f'A{rem_row}'].alignment = align_top_l
    set_outer_border(ws, rem_row, rem_row+4, 1, 10)

    # 保護
    ws.protection = SheetProtection(sheet=True, formatCells=False, formatColumns=False)
    ranges_to_unlock = ['B2:E2', 'G2:J2', 'B3:J3', 'B4', 'B5', f'A{start_row}:F{end_row}']
    ranges_to_unlock.append(f'A{rem_row}:J{rem_row+4}')
    for i in range(2, 8): # 解鎖更多 Seller Rows
        r = start_item_row + i
        ranges_to_unlock.append(f'A{r}')
        ranges_to_unlock.append(f'D{r}')
    for i in range(1, 8):
        r = start_item_row + i
        ranges_to_unlock.append(f'F{r}')
        ranges_to_unlock.append(f'J{r}')
    
    for rng_str in ranges_to_unlock:
        if ':' in rng_str:
            for row in ws[rng_str]:
                for cell in row:
                    cell.protection = Protection(locked=False)
        else:
            ws[rng_str].protection = Protection(locked=False)

    filename = '交屋租金分算表跨月可視化版.xlsx'
    wb.save(filename)
    return filename

if __name__ == '__main__':
    generate_expense_list_excel_v7_detailed()
